In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install -q "protobuf==3.20.3"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.0 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
pydrive2 1.21.3 requires cryptograp

In [1]:
# ============================================================
# 0. Install dependencies
# ============================================================
!pip install -q transformers datasets soundfile librosa accelerate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 38.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 55.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━

In [5]:
# ============================================================
# 1. Imports & paths
# ============================================================
import os
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
import random

import librosa
import torch

from datasets import Dataset, Audio
from sklearn.model_selection import train_test_split

from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

from dataclasses import dataclass
from typing import Any, Dict, List, Union

DATA_ROOT = Path("/kaggle/input/shobdotori")
print("Data root exists:", DATA_ROOT.exists())
print("Subfolders:", [p.name for p in DATA_ROOT.iterdir()])

# Fix random seeds for some stability
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"
device

Data root exists: True
Subfolders: ['sample_submission.csv', 'Train_annotation', 'Test', 'Train']


'cuda'

In [6]:
# ============================================================
# 2. Load & merge train annotations into a single DataFrame
# ============================================================
ann_root = DATA_ROOT / "Train_annotation"

all_ann_dfs = []
for csv_path in sorted(ann_root.glob("*.csv")):
    region = csv_path.stem  # e.g. "Barisal", "Noakhali"
    df = pd.read_csv(csv_path)
    df["region"] = region
    df["audio_path"] = df["audio"].apply(
        lambda x: str(DATA_ROOT / "Train" / region / x)
    )
    all_ann_dfs.append(df)

train_df = pd.concat(all_ann_dfs, ignore_index=True)
print("Train annotation shape:", train_df.shape)
train_df.head()


Train annotation shape: (3350, 4)


,audio,text,region,audio_path
0,female_barisal_1.wav,আজ সকালে আমি বাজারে গিয়েছিলাম।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
1,female_barisal_3.wav,আকাশে আজ মেঘ জমে আছে।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
2,female_barisal_4.wav,আমি বন্ধুদের সাথে খেলা খেলেছি।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
3,female_barisal_5.wav,দরজাটা ধীরে ধীরে বন্ধ করে দাও।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
4,female_barisal_6.wav,তুমি কি আমাকে পানি দিতে পারো?,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...


In [7]:
# ============================================================
# 3. Simple Bangla text normalization (for labels & predictions)
# ============================================================

def normalize_bn(text: str) -> str:
    """
    Very simple Bangla normalization:
    - strip leading/trailing whitespace
    - collapse multiple spaces
    - replace English '.' with Bangla '।'
    You can extend this later if needed.
    """
    if not isinstance(text, str):
        return ""
    text = text.strip()
    text = text.replace(".", "।")
    text = re.sub(r"\s+", " ", text)
    return text

# Normalize labels in train_df (important for consistent training)
train_df["text"] = train_df["text"].astype(str).apply(normalize_bn)
train_df.head()


,audio,text,region,audio_path
0,female_barisal_1.wav,আজ সকালে আমি বাজারে গিয়েছিলাম।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
1,female_barisal_3.wav,আকাশে আজ মেঘ জমে আছে।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
2,female_barisal_4.wav,আমি বন্ধুদের সাথে খেলা খেলেছি।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
3,female_barisal_5.wav,দরজাটা ধীরে ধীরে বন্ধ করে দাও।,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...
4,female_barisal_6.wav,তুমি কি আমাকে পানি দিতে পারো?,Barisal,/kaggle/input/shobdotori/Train/Barisal/female_...


In [30]:
# Train / validation split (10% val, stratified by region)
train_df_split, val_df_split = train_test_split(
    train_df,
    test_size=0.1,
    random_state=SEED,
    stratify=train_df["region"]
)

print("Train rows:", len(train_df_split))
print("Val rows:", len(val_df_split))


Train rows: 3015
Val rows: 335


In [31]:
model_name = "openai/whisper-medium"  # ⬅️ change here

processor = WhisperProcessor.from_pretrained(
    model_name,
    language="bengali",
    task="transcribe"
)

model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.to(device)

model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(
    language="bengali",
    task="transcribe"
)
model.config.suppress_tokens = []
model.config.use_cache = False


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [33]:
# ============================================================
# 6. Load Whisper-small processor & model
# ============================================================
model_name = "openai/whisper-small"

processor = WhisperProcessor.from_pretrained(
    model_name,
    language="bengali",
    task="transcribe"
)

model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.to(device)

# Set decoding config for Bangla transcription
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(
    language="bengali",
    task="transcribe"
)
model.config.suppress_tokens = []  # optional: don't suppress any tokens
model.config.use_cache = False     # important for Trainer (avoids warnings)

processor, model


In [15]:
# # ============================================================
# # 7. Preprocessing function: audio -> input_features, text -> labels
# # ============================================================

# import librosa

# def prepare_dataset(batch):
#     audio_path = batch["audio"]          # string path
#     audio_array, sr = librosa.load(audio_path, sr=16000)

#     # 1) Input features (audio → log-mel)
#     batch["input_features"] = processor(
#         audio_array,
#         sampling_rate=sr
#     ).input_features[0]

#     # 2) Labels (text → token ids)  ✅ no as_target_processor
#     text = batch["text"]
#     batch["labels"] = processor.tokenizer(
#         text,
#         add_special_tokens=True
#     ).input_ids

#     return batch


In [ ]:
# print(train_prepared[0].keys())
# train_prepared[0]

In [34]:
from torch.utils.data import Dataset, DataLoader #0.82345

class ASRDataset(Dataset):
    def __init__(self, df: pd.DataFrame, processor: WhisperProcessor):
        """
        df must contain: 'audio_path', 'text'
        """
        self.df = df.reset_index(drop=True)
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        audio_path = row["audio_path"]
        text = row["text"]

        # 1) Load audio
        audio_array, sr = librosa.load(audio_path, sr=16000)

        # 2) Compute input features (log-mel spectrogram)
        input_features = self.processor(
            audio_array,
            sampling_rate=16000
        ).input_features[0]  # shape (80, T)

        # 3) Tokenize labels
        labels = self.processor.tokenizer(
            text,
            add_special_tokens=True
        ).input_ids

        return {
            "input_features": input_features,
            "labels": labels,
        }

train_dataset = ASRDataset(train_df_split, processor)
val_dataset   = ASRDataset(val_df_split, processor)

len(train_dataset), len(val_dataset), train_dataset[0].keys()


(3015, 335, dict_keys(['input_features', 'labels']))

In [16]:
# from torch.utils.data import Dataset, DataLoader 
# class ASRDataset(Dataset):
#     def __init__(self, df: pd.DataFrame, processor: WhisperProcessor, augment: bool = False):
#         """
#         df must contain: 'audio_path', 'text'
#         augment=True  -> apply speed & noise augmentation
#         """
#         self.df = df.reset_index(drop=True)
#         self.processor = processor
#         self.augment = augment

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx: int):
#         row = self.df.iloc[idx]
#         audio_path = row["audio_path"]
#         text = row["text"]

#         audio_array, sr = librosa.load(audio_path, sr=16000)

#         # --------- AUGMENTATION (train only) ----------
#         if self.augment:
#             # Random speed: 0.9x, 1.0x, 1.1x
#             speed = random.choice([0.9, 1.0, 1.1])
#             if speed != 1.0:
#                 audio_array = librosa.effects.time_stretch(audio_array, rate=speed)

#             # Small Gaussian noise
#             noise_level = 0.003
#             noise = np.random.randn(len(audio_array)) * noise_level
#             audio_array = audio_array + noise
#         # ------------------------------------------------

#         input_features = self.processor(
#             audio_array,
#             sampling_rate=16000
#         ).input_features[0]

#         labels = self.processor.tokenizer(
#             text,
#             add_special_tokens=True
#         ).input_ids

#         return {"input_features": input_features, "labels": labels}


In [18]:
# train_dataset = ASRDataset(train_df_split, processor, augment=True)
# val_dataset   = ASRDataset(val_df_split,   processor, augment=False)

In [35]:
from dataclasses import dataclass
from typing import Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(
        self, features: List[Dict[str, Union[List[int], np.ndarray]]]
    ) -> Dict[str, torch.Tensor]:
        # Input features
        input_features = [
            {"input_features": f["input_features"]} for f in features
        ]
        # Label features
        label_features = [
            {"input_ids": f["labels"]} for f in features
        ]

        batch = self.processor.feature_extractor.pad(
            input_features,
            return_tensors="pt",
        )

        labels_batch = self.processor.tokenizer.pad(
            label_features,
            return_tensors="pt",
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["attention_mask"].ne(1), -100
        )

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [36]:
# batch_size = 4  # if OOM: use 2; if lots of memory: try 8
batch_size = 2  # if OOM: drop to 1

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

len(train_loader), len(val_loader)


(1508, 168)

In [37]:
def levenshtein_distance(a: str, b: str) -> int:
    n, m = len(a), len(b)
    if n == 0:
        return m
    if m == 0:
        return n

    dp = [[0] * (m + 1) for _ in range(n + 1)]
    for i in range(n + 1):
        dp[i][0] = i
    for j in range(m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = 0 if a[i - 1] == b[j - 1] else 1
            dp[i][j] = min(
                dp[i - 1][j] + 1,          # deletion
                dp[i][j - 1] + 1,          # insertion
                dp[i - 1][j - 1] + cost    # substitution
            )
    return dp[n][m]

def normalized_similarity(pred: str, ref: str) -> float:
    pred = normalize_bn(pred)
    ref = normalize_bn(ref)
    if len(pred) == 0 and len(ref) == 0:
        return 1.0
    max_len = max(len(pred), len(ref))
    if max_len == 0:
        return 1.0
    dist = levenshtein_distance(pred, ref)
    sim = 1.0 - dist / max_len
    return sim


In [38]:
def evaluate_model(model, val_loader, processor, device, max_length=128):
    model.eval()
    sims = []

    with torch.no_grad():
        for batch in val_loader:
            input_features = batch["input_features"].to(device)
            labels = batch["labels"].to(device)

            # Generate predictions
            # generated_ids = model.generate(
            #     input_features,
            #     max_length=max_length
            # )
            generated_ids = model.generate(
                input_features,
                max_length=128,
                num_beams=5,
                early_stopping=True
            )


            # Decode predictions
            pred_str = processor.tokenizer.batch_decode(
                generated_ids,
                skip_special_tokens=True
            )

            # Decode labels (replace -100 with pad_token_id)
            labels_clone = labels.clone()
            labels_clone[labels_clone == -100] = processor.tokenizer.pad_token_id
            label_str = processor.tokenizer.batch_decode(
                labels_clone,
                skip_special_tokens=True
            )

            for p, r in zip(pred_str, label_str):
                sims.append(normalized_similarity(p, r))

    mean_sim = float(np.mean(sims)) if sims else 0.0
    return mean_sim


In [39]:
from torch.optim import AdamW

learning_rate = 1e-5
# optimizer = AdamW(model.parameters(), lr=learning_rate)
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

# num_epochs = 8  # start with 3; can increase later
num_epochs = 4  # or 4 max for a first run

best_val_sim = 0.0
best_model_path = "./whisper-small-shobdotori-manual"

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0.0
    step = 0

    for batch in train_loader:
        step += 1

        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_features=input_features,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if step % 50 == 0:
            print(f"Epoch {epoch} | Step {step} | Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / max(1, step)
    print(f"\nEpoch {epoch} finished. Avg train loss: {avg_train_loss:.4f}")

    # Evaluate
    val_sim = evaluate_model(model, val_loader, processor, device)
    print(f"Epoch {epoch} | Validation mean_norm_levenshtein: {val_sim:.4f}")

    # Save best
    if val_sim > best_val_sim:
        best_val_sim = val_sim
        print(f"New best model! Saving to {best_model_path}")
        model.save_pretrained(best_model_path)
        processor.save_pretrained(best_model_path)

print(f"\nTraining done. Best val similarity: {best_val_sim:.4f}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 3274 has 14.73 GiB memory in use. Of the allocated memory 13.75 GiB is allocated by PyTorch, and 863.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [24]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

finetuned_processor = WhisperProcessor.from_pretrained(best_model_path)
finetuned_model = WhisperForConditionalGeneration.from_pretrained(best_model_path).to(device)

finetuned_model.config.forced_decoder_ids = finetuned_processor.get_decoder_prompt_ids(
    language="bengali",
    task="transcribe"
)
finetuned_model.eval()


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [25]:
def transcribe_finetuned(audio_path: str) -> str:
    audio_array, sr = librosa.load(audio_path, sr=16000)
    inputs = finetuned_processor(
        audio_array,
        sampling_rate=16000,
        return_tensors="pt"
    )
    input_features = inputs.input_features.to(device)

    with torch.no_grad():
        generated_ids = finetuned_model.generate(
            input_features,
            max_length=128
        )

    pred = finetuned_processor.tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True
    )[0]

    pred = normalize_bn(pred)
    return pred


In [26]:
test_root = DATA_ROOT / "Test"
test_files = sorted(test_root.glob("*.wav"))

print("Number of test files:", len(test_files))
print("First few:", [p.name for p in test_files[:10]])

pred_rows = []
for wav_path in tqdm(test_files):
    file_name = wav_path.name
    pred_text = transcribe_finetuned(str(wav_path))
    pred_rows.append({
        "audio": file_name,
        "text": pred_text
    })

submission_df = pd.DataFrame(pred_rows)
submission_df = submission_df.sort_values("audio")
submission_df.head()


Number of test files: 450
First few: ['test_001.wav', 'test_002.wav', 'test_003.wav', 'test_004.wav', 'test_005.wav', 'test_006.wav', 'test_007.wav', 'test_008.wav', 'test_009.wav', 'test_010.wav']


  0%|          | 0/450 [00:00<?, ?it/s]`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}. If this is not desired, please set these values explicitly.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_p

,audio,text
0,test_001.wav,তুমি কি আগামে গল্প করবে?
1,test_002.wav,তুমি কি আমাকে কলমটা দেবে?
2,test_003.wav,আজ দুপুরে রাস্তায় পানি জমেছে।
3,test_004.wav,আজকের সকালে হাঁটা খেতে বসেছে।
4,test_005.wav,তুমি কি মোবাইল করতে যাচ্ছো?


In [28]:
submission_path = Path("submission.csv")
submission_df.to_csv(submission_path, index=False, encoding="utf-8")
print("Saved submission to:", submission_path.resolve())

submission_df.head(20)


Saved submission to: /kaggle/working/submission.csv


,audio,text
0,test_001.wav,তুমি কি আগামে গল্প করবে?
1,test_002.wav,তুমি কি আমাকে কলমটা দেবে?
2,test_003.wav,আজ দুপুরে রাস্তায় পানি জমেছে।
3,test_004.wav,আজকের সকালে হাঁটা খেতে বসেছে।
4,test_005.wav,তুমি কি মোবাইল করতে যাচ্ছো?
5,test_006.wav,তুমি কি মোবাইল করণ্ড হয়েছো?
6,test_007.wav,আজকের দোকে মেঘ জম পরে ক্রিকেন।
7,test_008.wav,আমার ছোট ভাই স্কুলে যাচ্ছে।
8,test_009.wav,তুমি কি গান গাইলে গাইলে গাইলে?
9,test_010.wav,তুমি কি আজ বন্ধুর সাথে দেখা করেছো?


In [29]:
import os

print("CWD:", os.getcwd())
print("Files here:", os.listdir())


CWD: /kaggle/working
Files here: ['whisper-small-shobdotori-manual', '.virtual_documents', 'submission.csv']


In [39]:
!ls -al


total 56
drwxr-xr-x 4 root root  4096 Nov 16 10:28 .
drwxr-xr-x 5 root root  4096 Nov 16 05:32 ..
-rw-r--r-- 1 root root 40596 Nov 16 10:37 submission.csv
drwxr-xr-x 2 root root  4096 Nov 16 05:32 .virtual_documents
drwxr-xr-x 2 root root  4096 Nov 16 06:20 whisper-small-shobdotori-manual
